<a href="https://colab.research.google.com/github/gfarhani/Quantum_Computing/blob/main/Chapter_4/Quantum_Phase_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 103.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit_aer import Aer
from qiskit import transpile

# ----- Choose a phase you want to estimate -----
phi = 3/8  # try 1/8, 3/8, 5/8, etc. (exactly representable in 3 bits)

n_count = 3  # number of counting qubits (precision ~ 1/2^n_count)

qc = QuantumCircuit(n_count + 1, n_count)

# Target (eigenstate) qubit is the last one
target = n_count

# Prepare |1> on target (eigenstate of P(2πφ))
qc.x(target)

# Put counting register in |+...+>
qc.h(range(n_count))

# Controlled-U^(2^k) applications
# For U = P(2πφ), U^(2^k) = P(2πφ * 2^k)
for k in range(n_count):
    angle = 2 * np.pi * phi * (2**k)
    qc.cp(angle, k, target)   # controlled-phase

# Inverse QFT on counting register
qc.append(QFT(n_count, inverse=True, do_swaps=False).to_gate(), range(n_count))

# Measure counting qubits
qc.measure(range(n_count), range(n_count))

print(qc.draw())

# Simulate
sim = Aer.get_backend("aer_simulator")
tqc = transpile(qc, sim)
result = sim.run(tqc, shots=2000).result()
counts = result.get_counts()

print("Counts:", counts)

# Most likely bitstring corresponds to phi in binary with n_count bits
best = max(counts, key=counts.get)
est = int(best, 2) / (2**n_count)
print("Most likely bitstring:", best)
print("Estimated phi:", est, " (true phi:", phi, ")")


/tmp/ipython-input-1046524793.py:30: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_count, inverse=True, do_swaps=False).to_gate(), range(n_count))


     ┌───┐                            ┌───────┐┌─┐      
q_0: ┤ H ├─■──────────────────────────┤0      ├┤M├──────
     ├───┤ │                          │       │└╥┘┌─┐   
q_1: ┤ H ├─┼─────────■────────────────┤1 IQFT ├─╫─┤M├───
     ├───┤ │         │                │       │ ║ └╥┘┌─┐
q_2: ┤ H ├─┼─────────┼─────────■──────┤2      ├─╫──╫─┤M├
     ├───┤ │P(3π/4)  │P(3π/2)  │P(3π) └───────┘ ║  ║ └╥┘
q_3: ┤ X ├─■─────────■─────────■────────────────╫──╫──╫─
     └───┘                                      ║  ║  ║ 
c: 3/═══════════════════════════════════════════╩══╩══╩═
                                                0  1  2 
Counts: {'100': 145, '010': 81, '110': 73, '111': 251, '011': 1450}
Most likely bitstring: 011
Estimated phi: 0.375  (true phi: 0.375 )
